In [16]:
!pip install -q ja_ginza_electra pandarallel

In [1]:
# 前処理用
import re
from tqdm.auto import tqdm
tqdm.pandas()
import os
import pandas as pd
import numpy as np
import emoji
import spacy
import re
import neologdn
import json

# 分かち書き用
import ginza
import ja_ginza_electra
from pandarallel import pandarallel # ライブラリー
pandarallel.initialize(progress_bar=True)

train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
def wakati_rm_func(x):
    nlp = spacy.load('ja_ginza_electra')
    sentence = x
    
    sentence = re.sub(r'[!-~]'," ",sentence,) # 小文字の記号を削除
    sentence=re.sub(r'[︰-＠]', "", sentence) # 大文字の記号を削除
    # 絵文字削除
    sentence = ''.join(['' if c in emoji.UNICODE_EMOJI else c for c in sentence])

    # 不要記号削除
    pattern = '[!"#$%&\'\\\\()*+,-./:;<=>?@[\\]^_`{|}~「」〔〕“”◇ᴗ●↓→♪★⊂⊃※△□◎〈〉『』【】＆＊・（）＄＃＠。、？！｀＋￥％�]'
    sentence =  re.sub(pattern, ' ', sentence)

    # 正規化する
    sentence = neologdn.normalize(sentence)
 

    # 大文字・小文字変換
    sentence = sentence.lower()
    sentence = sentence.replace("\n", "")
    # GinZaで分かち書きをする
    doc = nlp(sentence)
    nlp = spacy.load('ja_ginza_electra')
    tmp_words_list = []
    for sent in doc.sents:
        for token in sent:
            if token.pos_ in ["PROPN","NOUN", "ADJ", "VERB"]:
                tmp_words_list.append(token.orth_)

    result = " ".join(tmp_words_list)
    return result

In [3]:
# 試すために数を減らす
train = train.head(10)
test = test.head(10)

# trainに分かち書きを実行する
train["story_wakati"] = train["story"].progress_apply(wakati_rm_func)
train["title_wakati"] = train["title"].progress_apply(wakati_rm_func)
train["keyword_wakati"] = train["keyword"].progress_apply(wakati_rm_func)

# testに分かち書きを実行する
test["story_wakati"] = test["story"].progress_apply(wakati_rm_func)
test["title_wakati"] = test["title"].progress_apply(wakati_rm_func)
test["keyword_wakati"] = test["keyword"].progress_apply(wakati_rm_func)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(train["story_wakati"])
count_vec_story_train = pd.DataFrame(data= X.toarray(), columns = vectorizer.get_feature_names())

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(train["title_wakati"])
count_vec_title_train = pd.DataFrame(data= X.toarray(), columns = vectorizer.get_feature_names())

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(train["keyword_wakati"])
count_vec_keyword_train = pd.DataFrame(data= X.toarray(), columns = vectorizer.get_feature_names())

pd.concat([count_vec_story_train, count_vec_title_train, count_vec_keyword_train]).to_pickle("../data/train_countvec.pkl")

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(test["story_wakati"])
count_vec_story_test = pd.DataFrame(data= X.toarray(), columns = vectorizer.get_feature_names())

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(test["title_wakati"])
count_vec_title_test = pd.DataFrame(data= X.toarray(), columns = vectorizer.get_feature_names())

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(test["keyword_wakati"])
count_vec_keyword_test = pd.DataFrame(data= X.toarray(), columns = vectorizer.get_feature_names())

pd.concat([count_vec_story_test, count_vec_title_test, count_vec_keyword_test]).to_pickle("../data/test_countvec.pkl")

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
model = TfidfVectorizer(max_df=0.9)
X = model.fit_transform(train["story_wakati"])
story_tfidf_train = pd.DataFrame(data= X.toarray(), columns = model.get_feature_names())

model = TfidfVectorizer(max_df=0.9)
X = model.fit_transform(train["title_wakati"])
title_tfidf_train = pd.DataFrame(data= X.toarray(), columns = model.get_feature_names())

model = TfidfVectorizer(max_df=0.9)
X = model.fit_transform(train["keyword_wakati"])
keyword_tfidf_train = pd.DataFrame(data= X.toarray(), columns = model.get_feature_names())

pd.concat([story_tfidf_train, title_tfidf_train, keyword_tfidf_train]).to_pickle("../data/train_tfidf.pkl")

model = TfidfVectorizer(max_df=0.9)
X = model.fit_transform(test["story_wakati"])
story_tfidf_test = pd.DataFrame(data= X.toarray(), columns = model.get_feature_names())

model = TfidfVectorizer(max_df=0.9)
X = model.fit_transform(test["title_wakati"])
title_tfidf_test = pd.DataFrame(data= X.toarray(), columns = model.get_feature_names())

model = TfidfVectorizer(max_df=0.9)
X = model.fit_transform(test["keyword_wakati"])
keyword_tfidf_test = pd.DataFrame(data= X.toarray(), columns = model.get_feature_names())

pd.concat([story_tfidf_test, title_tfidf_test, keyword_tfidf_test]).to_pickle("../data/test_tfidf.pkl")

,アリアドネ,クライムズクライシス,ゲームヴァルハラ,ピーターパン,仮想現実,向こう,夢幻,宇宙,憤悶,時代,暗い,異伝,皇帝,窓辺,竜騎兵,自由,誘い,遠く
0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.57735,0.000000,0.57735,0.57735,0.0,0.000000,0.000000,0.000000
1,0.000000,0.0,0.707107,0.0,0.707107,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000,0.0,0.000000,0.000000,0.000000
2,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000,1.0,0.000000,0.000000,0.000000
3,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.707107,0.000000,0.707107,0.00000,0.000000,0.00000,0.00000,0.0,0.000000,0.000000,0.000000
4,0.000000,1.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000,0.0,0.000000,0.000000,0.000000
5,0.000000,0.0,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000,0.0,0.000000,0.000000,0.000000
6,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.707107,0.000000,0.000000,0.000000,0.00000,0.707107,0.00000,0.00000,0.0,0.000000,0.000000,0.000000
7,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000,0.0,0.707107,0.707107,0.000000
8,0.000000,0.0,0.000000,0.0,0.000000,0.707107,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000,0.0,0.000000,0.000000,0.707107
9,0.707107,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.707107,0.000000,0.00000,0.000000,0.00000,0.00000,0.0,0.000000,0.000000,0.000000
